<a href="https://colab.research.google.com/github/tswells78/CHFI-Trivia/blob/master/chfi_trivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import re
from random import randint
import textwrap

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1_glUNcVXQWJFOkz3k640kWW2gUCPzuxu'

In [13]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1_glUNcVXQWJFOkz3k640kWW2gUCPzuxu


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('312-49-CHFI_merged_sets.xlsx')  
question_count = 0
score = 0

In [0]:
def how_many_questions():
  return input('How many questions do you want to try?  -->  ')

In [0]:
def question_data():
  sheet_num = randint(1,5)
  chfi = pd.read_excel('312-49-CHFI_merged_sets.xlsx', sheet_name='Set %s' % str(sheet_num))

  chfi = chfi.fillna('None')

  questions = [x for x in chfi.Questions if 'QUESTION NO' in x]
  a_answers = [x for x in chfi.Questions if re.match('^A\. .+', x)]
  b_answers = [x for x in chfi.Questions if re.match('^B\. .+', x)]
  c_answers = [x for x in chfi.Questions if re.match('^C\. .+', x)]
  d_answers = [x for x in chfi.Questions if re.match('^D\. .+', x)]
  answers = [x for x in chfi.Questions if re.match('^Answer: .+', x)]

  question_set = pd.DataFrame({'Qusetions': questions, 'A Answers': a_answers,
                                  'B Answers': b_answers, 'C Answers': c_answers,
                                  'D Answers': d_answers, 'Answers': answers})

  return question_set

In [0]:
def get_question():
  random_question_number = randint(0,100)
  return question_data().loc[random_question_number].values

In [0]:
def score_card():
  global score, question_count
  final_score: str = "{:.00%}".format(float(score / question_count))
  output = f'You scored a {final_score}'
  return output

In [0]:
def main():
  global score
  try:
    quiz_item = get_question().tolist()
  except KeyError:
    quiz_item = get_question().tolist()

  answer = quiz_item[5].split(':')[1].strip()

  wrapper = textwrap.TextWrapper(width=50)
  the_question = wrapper.wrap(text=quiz_item[0])
  for lines in the_question:
    print(lines)
  print()

  for a in quiz_item[1:5]:
    print(a)
  print()

  user_answer = input('Answer ->  ').capitalize()
  if user_answer == answer:
    score += 1
    print('Correct')
  else:
    print(f'Sorry it was {answer}')

In [0]:
try:
  number_of_questions = int(how_many_questions())
except ValueError:
  print('Must be a number')
  number_of_questions = int(how_many_questions())

print()
try:
  while question_count != number_of_questions:
    main()
    question_count += 1
    print(f'You are {score} for {question_count} of {number_of_questions}\n')
except KeyboardInterrupt:
  print('Quitter!')
  print(score_card())

print(score_card())